In [1]:
import sys
sys.path.append("../")
from utils import load_cnf, cnf_to_bqm
from embedding_utils import *
import seaborn as sns
from redundancy import *
from utils import *
from dwave.embedding.chain_strength import scaled
import pandas as pd
from brute_force import get_bqm_random_sat
import numpy as np
import matplotlib.pyplot as plt
from statistics import variance

In [2]:
pegasus_qpu = get_qpu()
zephyr_qpu = dnx.zephyr_graph(15)

In [3]:
def get_vertices_degrees(bqm):
    answer = []
    for (node, neighbours) in bqm.adj.items():
        answer.append(len(neighbours)) 
    return answer

In [4]:
var_counts = []
clauses_counts = []
ratios_c_to_var = []
avg_qubo_connectivity = []
variances_qubo_connectivity = []
binary_variables_counts = []
for i in range(250, 1001, 50):
    for j in range(1, 11):
        
        # process CNF
        path = f"./hgen2_instances/hgen_{i}_{j}.cnf"
        print(path)
        num_variables, num_clauses, variables, clauses = load_cnf(path)
        
        # build QUBO
        bqm, or_result_vars, clauses_qubits, _ = cnf_to_bqm(variables, clauses)
        
        vertices_degrees = get_vertices_degrees(bqm)
        avg_degrees = float(sum(vertices_degrees))/float(len(vertices_degrees))
        variance_degrees = (variance(vertices_degrees))
        
        assert(num_variables == i)
        
        ratio = float(num_clauses)/float(num_variables)
        
        var_counts.append(num_variables)
        clauses_counts.append(num_clauses)
        ratios_c_to_var.append(round(ratio, 2))
        avg_qubo_connectivity.append(round(avg_degrees, 2))
        variances_qubo_connectivity.append(round(variance_degrees, 2))
        binary_variables_counts.append(len(bqm.variables))
        

./hgen2_instances/hgen_250_1.cnf
./hgen2_instances/hgen_250_2.cnf
./hgen2_instances/hgen_250_3.cnf
./hgen2_instances/hgen_250_4.cnf
./hgen2_instances/hgen_250_5.cnf
./hgen2_instances/hgen_250_6.cnf
./hgen2_instances/hgen_250_7.cnf
./hgen2_instances/hgen_250_8.cnf
./hgen2_instances/hgen_250_9.cnf
./hgen2_instances/hgen_250_10.cnf
./hgen2_instances/hgen_300_1.cnf
./hgen2_instances/hgen_300_2.cnf
./hgen2_instances/hgen_300_3.cnf
./hgen2_instances/hgen_300_4.cnf
./hgen2_instances/hgen_300_5.cnf
./hgen2_instances/hgen_300_6.cnf
./hgen2_instances/hgen_300_7.cnf
./hgen2_instances/hgen_300_8.cnf
./hgen2_instances/hgen_300_9.cnf
./hgen2_instances/hgen_300_10.cnf
./hgen2_instances/hgen_350_1.cnf
./hgen2_instances/hgen_350_2.cnf
./hgen2_instances/hgen_350_3.cnf
./hgen2_instances/hgen_350_4.cnf
./hgen2_instances/hgen_350_5.cnf
./hgen2_instances/hgen_350_6.cnf
./hgen2_instances/hgen_350_7.cnf
./hgen2_instances/hgen_350_8.cnf
./hgen2_instances/hgen_350_9.cnf
./hgen2_instances/hgen_350_10.cnf
./hgen2

In [5]:
df = pd.DataFrame({
    'vars': var_counts,
    'clauses': clauses_counts,
    "clauses_per_var": ratios_c_to_var,
    "qubo_variables": binary_variables_counts,
    'qubo_avg_conn': avg_qubo_connectivity,
    'qubo_variance_conn': variances_qubo_connectivity
})
df

,vars,clauses,clauses_per_var,qubo_variables,qubo_avg_conn,qubo_variance_conn
0,250,875,3.5,1125,6.22,37.12
1,250,875,3.5,1125,6.22,37.06
2,250,875,3.5,1125,6.22,37.08
3,250,875,3.5,1125,6.22,37.09
4,250,875,3.5,1125,6.22,37.10
...,...,...,...,...,...,...
155,1000,3500,3.5,4500,6.22,37.12
156,1000,3500,3.5,4500,6.22,37.16
157,1000,3500,3.5,4500,6.22,37.13
158,1000,3500,3.5,4500,6.22,37.12


In [7]:
df.to_csv('attributes_hgen_formulas.csv')

In [8]:
df.describe()

,vars,clauses,clauses_per_var,qubo_variables,qubo_avg_conn,qubo_variance_conn
count,160.000000,160.000000,160.0,160.00000,160.00,160.000000
mean,625.000000,2187.500000,3.5,2812.50000,6.22,37.108125
std,231.212282,809.242988,0.0,1040.45527,0.00,0.042117
min,250.000000,875.000000,3.5,1125.00000,6.22,37.000000
25%,437.500000,1531.250000,3.5,1968.75000,6.22,37.080000
50%,625.000000,2187.500000,3.5,2812.50000,6.22,37.110000
75%,812.500000,2843.750000,3.5,3656.25000,6.22,37.140000
max,1000.000000,3500.000000,3.5,4500.00000,6.22,37.220000


In [9]:
df.qubo_variables.unique()

array([1125, 1350, 1575, 1800, 2025, 2250, 2475, 2700, 2925, 3150, 3375,
       3600, 3825, 4050, 4275, 4500])

## testing the embedding on a formula

In [3]:
path = f"./hgen2_instances/hgen_250_7.cnf"
print(path)
num_variables, num_clauses, variables, clauses = load_cnf(path)

bqm, or_result_vars, clauses_qubits, _ = cnf_to_bqm(variables, clauses)

./hgen2_instances/hgen_250_7.cnf


In [ ]:
embedding_p = get_embedding(bqm, pegasus_qpu)
get_embedding_statistics(embedding_p)

In [5]:
embedding_z = get_embedding(bqm, zephyr_qpu, isnetworkx=True)
get_embedding_statistics(embedding_z)

ValueError: max() arg is an empty sequence